# 네이버 검색 api 서비스
# 라이브러리 불러오기

In [1]:
import numpy as np
import pandas as pd
import os
import glob
import seaborn as sns
sns.set_style('whitegrid', {'grid.linestyle':'--'})
import matplotlib
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')
import datetime as dt
import warnings 
warnings.filterwarnings('ignore') 

In [2]:
def get_font_family():
    """
    시스템 환경에 따른 기본 폰트명을 반환하는 함수
    """
    import platform
    system_name = platform.system()

    if system_name == "Darwin" :
        font_family = "AppleGothic"
    elif system_name == "Windows":
        font_family = "Malgun Gothic"
    else:
        # Linux(Colab)
        !apt-get install fonts-nanum -qq  > /dev/null
        !fc-cache -fv

        import matplotlib as mpl
        mpl.font_manager._rebuild()
        findfont = mpl.font_manager.fontManager.findfont
        mpl.font_manager.findfont = findfont
        mpl.backends.backend_agg.findfont = findfont
        
        font_family = "NanumBarunGothic"
    return font_family

plt.rc("font", family=get_font_family())
plt.rc("axes", unicode_minus=False)

In [3]:
pd.options.display.max_columns=None

# 네이버 웹문서 서칭 api

In [5]:
import os
import sys
import urllib.request
import json
import re # 정규표현식

client_id = "--"
client_secret = "--"

query = urllib.parse.quote(input('검색 질의 : '))

idx = 0 # 번호남기기
display = 100 # 몇개씩보여줄래
start = 1
end = 1000 # 몇개씩가져올지

web_df = pd.DataFrame(columns = ('Title', 'Link', 'Description'))# 저장

for start_index in range(start, end, display): # 100개씩 가져온다.

    url = "https://openapi.naver.com/v1/search/webkr?query=" + query +"&display="  + str(display)+"&start=" + str(start_index) 
#         +"&display= "  + str(display)\ 
#         +"&start= " + str(start_index)
        
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode==200): # 호출성공이200
        response_body = response.read()
        response_dict = json.loads(response_body.decode('utf-8'))
        items = response_dict['items']
        for item_index in range(0, len(items)):
            remove_tag = re.compile('<.*?>')
            title = re.sub(remove_tag, '', items[item_index]['title'])
            link = items[item_index]['link']
            description = re.sub(remove_tag, '', items[item_index]['description'])
            web_df.loc[idx] = [title, link, description]
            idx += 1
    else:
        print("Error Code:" + rescode)
web_df

검색 질의 :  애완동물


,Title,Link,Description
0,애완동물 - 나무위키,https://namu.wiki/w/%EC%95%A0%EC%99%84%EB%8F%9...,애완동물(愛玩動物)은 좋아하여 가까이 두고 귀여워하며 기르는 동물을 말한다. 엄밀하...
1,여행/취미/레저,http://browse.gmarket.co.kr/list?category=3000...,"유쾌한수의사의동물병원24시 14,400원 6/3(금) 도착예정 카드 15% (7) ..."
2,애완동물 | Air France - 에어프랑스 코리아,https://wwws.airfrance.co.kr/information/passa...,애완 동물과 함께 항공 여행을 하시려고 합니까 ? 6kg 이하의 개와 고양이만이 위...
3,분류:애완동물 - 나무위키,https://namu.wiki/w/%EB%B6%84%EB%A5%98:%EC%95%...,이 분류에 대한 설명은 애완동물 문서 를 참고하십시오.
4,애완동물 - 위키하우의 ~하는 방법 글,https://ko.wikihow.com/%EB%B6%84%EB%A5%98:%EC%...,"위키하우는 애완동물 단계적 설명과 사진이 있는 ~하는 법 글을 다룬다. 고양이, 개..."
...,...,...,...
145,애완동물 - ShindanMaker,https://kr.shindanmaker.com/213648,당신에세 알맞는 애완동물은?
146,애완동물 - 일본친구와 감성을 나누는 즐거운 세상! - 고재팬,http://www.gojapan.com/trans_bulletin/photo_li...,* 게시글 삭제 및 이용제한 안내 애완동물 NO. Title Name Date Hi...
147,"서울종합예술실용학교(SAC) 애완동물계열, 2020년도 신입생 추가 모집 | 올치올치",https://www.olchiolchi.com/%EC%84%9C%EC%9A%B8%...,"서울종합예술실용학교(이하 서종예, SAC) 애완동물계열에서 2020년도 신입생 추가..."
148,애완동물 천도 재 - 뉴스프리존,http://www.newsfreezone.co.kr/news/articleView...,애완동물 천도 재우리는 죽어서 어디로 갈까요? 누구나 살면서 한번쯤 죽음을 생각하게...


- 우리가원하는 웹문서의 정보는
    - title
    - link
    - description
- 우리가 원하는정보를 parsing해야한다.

# 네이버 뉴스 검색 api
https://developers.naver.com/docs/serviceapi/search/news/news.md#%EB%89%B4%EC%8A%A4

In [40]:
import os
import sys
import urllib.request
import json
import re # 정규표현식

client_id = "--"
client_secret = "--"

query = urllib.parse.quote(input('검색 질의 : '))

idx = 0 # 번호남기기
display = 100 # 몇개씩보여줄래
start = 1
end = 1000 # 몇개씩가져올지
sort = 'sim'

news_df = pd.DataFrame(columns = ('Title', 'Original Link', 'Link', 'Description', 'Publication Date'))# 저장

for start_index in range(start, end, display): # 100개씩 가져온다.

    url = "https://openapi.naver.com/v1/search/news?query=" + query +"&display="  + str(display) + "&start=" + str(start_index) + "&sort=" + sort  
        
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode==200): # 호출성공이200
        response_body = response.read()
        response_dict = json.loads(response_body.decode('utf-8'))
        items = response_dict['items']
        for item_index in range(0, len(items)):
            remove_tag = re.compile('<.*?>')
            title = re.sub(remove_tag, '', items[item_index]['title'])
            original_link = items[item_index]['originallink']
            link = items[item_index]['link']            
            description = re.sub(remove_tag, '', items[item_index]['description'])
            pub_date = items[item_index]['pubDate']

            news_df.loc[idx] = [title, original_link, link, description, pub_date]
            idx += 1
    else:
        print("Error Code:" + rescode)
news_df

검색 질의 :  댕댕


,Title,Original Link,Link,Description,Publication Date
0,"'댕댕美' 헤이즈, 오늘도 리즈찍은 화보... 태연 &quot;이야~~&quot;[...",http://www.sportsseoul.com/news/read/1137085?r...,https://n.news.naver.com/mnews/article/468/000...,가수 헤이즈. 출처 | 보그 ‘음색여신’ 헤이즈가 장꾸력이 돋보이는 화보를 공개했다...,"Tue, 28 Jun 2022 11:12:00 +0900"
1,[쿠킹] 댕댕푸드대첩 우승…강아지 입맛 사로잡을 초간단 레시피,https://www.joongang.co.kr/article/25082630,https://n.news.naver.com/mnews/article/025/000...,"지난달 21일, 고양시 킨텍스에서 열린 펫푸드 요리대회 ‘제1회 댕댕푸드 대첩’엔 ...","Tue, 28 Jun 2022 09:00:00 +0900"
2,"&quot;올 여름 휴가는 댕댕 냥냥 함께 가세요&quot; 현대차 펫 프랜들리, ...",http://www.kdfnews.com/news/articleView.html?i...,http://www.kdfnews.com/news/articleView.html?i...,반려동물을 키우는 인구가 1500만명에 달하면서 '휴가는 반려동물과 함께'라는 트랜...,"Tue, 28 Jun 2022 10:00:00 +0900"
3,&quot;반려동물과 휴가 오세요&quot;…'2022 댕댕 페스타 in 강원' 홍...,http://www.kado.net/news/articleView.html?idxn...,https://n.news.naver.com/mnews/article/654/000...,유명 동물훈련사인 강형욱이 참여하는 '2022 댕댕 페스타 in 강원'이 홍천 소노...,"Sat, 25 Jun 2022 10:16:00 +0900"
4,‘2022 댕댕 페스타 in 강원’ 내일까지 홍천에서,https://news.kbs.co.kr/news/view.do?ncd=549438...,https://n.news.naver.com/mnews/article/056/001...,강원도관광재단이 마련한 '2022 댕댕 페스타 in 강원'이 오늘(24일) 홍천 소...,"Fri, 24 Jun 2022 21:58:00 +0900"
...,...,...,...,...,...
995,"방탄소년단 정국, 비눗방울에 세상 신난 美소년 &quot;몽글 순둥 사랑둥이♥&quot;",http://star.mt.co.kr/stview.php?no=20211113070...,https://n.news.naver.com/mnews/article/108/000...,정국은 여심을 녹일 몽글몽글한 댕댕미와 상큼 청량한 소년의 매력을 동시에 발산했고 ...,"Sat, 13 Nov 2021 07:08:00 +0900"
996,"&quot;몽글 순둥 사랑둥이♥&quot; 방탄소년단 정국, 아이 같은 순수 미소X...",http://www.topstarnews.net/news/articleView.ht...,http://www.topstarnews.net/news/articleView.ht...,정국은 여심을 녹일 몽글몽글한 댕댕미와 상큼 청량한 소년의 매력을 동시에 발산했고 ...,"Sat, 13 Nov 2021 09:40:00 +0900"
997,"[HD포토] 워너원(Wanna One) 김재환, '댕댕美 뿜뿜' (2018 코리아 ...",http://www.topstarnews.net/news/articleView.ht...,http://www.topstarnews.net/news/articleView.ht...,8월 1일 서울 구로구 고척스카이돔에서 '케이스타 2018 코리아 뮤직 페스티벌'이...,"Wed, 01 Aug 2018 22:23:00 +0900"
998,‘댕댕아 함께 가자’ 반려견과 떠나는 여행지 280곳,http://www.hankookilbo.com/v/f91ead099f2b454a9...,https://n.news.naver.com/mnews/article/469/000...,"혼자 두고 가자니 미안하고, 남의 손에 맡기자니 찝찝하다. 그렇다고 함께 가자니 문...","Wed, 11 Jul 2018 15:53:00 +0900"


In [41]:
news_df['Publication Date'][1][12:16]

'2022'

In [42]:
news_df['년'] = news_df['Publication Date'].apply(lambda x : x[12:16])

In [43]:
news_title = news_df[['Title', '년']]
news_title

,Title,년
0,"'댕댕美' 헤이즈, 오늘도 리즈찍은 화보... 태연 &quot;이야~~&quot;[...",2022
1,[쿠킹] 댕댕푸드대첩 우승…강아지 입맛 사로잡을 초간단 레시피,2022
2,"&quot;올 여름 휴가는 댕댕 냥냥 함께 가세요&quot; 현대차 펫 프랜들리, ...",2022
3,&quot;반려동물과 휴가 오세요&quot;…'2022 댕댕 페스타 in 강원' 홍...,2022
4,‘2022 댕댕 페스타 in 강원’ 내일까지 홍천에서,2022
...,...,...
995,"방탄소년단 정국, 비눗방울에 세상 신난 美소년 &quot;몽글 순둥 사랑둥이♥&quot;",2021
996,"&quot;몽글 순둥 사랑둥이♥&quot; 방탄소년단 정국, 아이 같은 순수 미소X...",2021
997,"[HD포토] 워너원(Wanna One) 김재환, '댕댕美 뿜뿜' (2018 코리아 ...",2018
998,‘댕댕아 함께 가자’ 반려견과 떠나는 여행지 280곳,2018


In [44]:
news_title.to_csv('./data/네이버api/new_(0628)naver_댕댕.csv', encoding = 'utf-8')

# 네이버 블로그 검색 api
https://developers.naver.com/docs/serviceapi/search/blog/blog.md#%EB%B8%94%EB%A1%9C%EA%B7%B8

In [37]:
client_id = "--"
client_secret = "--"

query = urllib.parse.quote(input('검색 질의 : '))

idx = 0 # 번호남기기
display = 100 # 몇개씩보여줄래
start = 1
end = 1000 # 몇개씩가져올지
sort = 'sim'

blog_df = pd.DataFrame(columns = ('Title', 'Link', 'Description', 'Blogger Name', 'Blogger Link','postdate'))# 저장

for start_index in range(start, end, display): # 100개씩 가져온다.

    url = "https://openapi.naver.com/v1/search/blog?query=" + query +"&display="  + str(display) + "&start=" + str(start_index) + "&sort=" + sort  
        
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode==200): # 호출성공이200
        response_body = response.read()
        response_dict = json.loads(response_body.decode('utf-8'))
        items = response_dict['items']
        for item_index in range(0, len(items)):
            remove_tag = re.compile('<.*?>')
            title = re.sub(remove_tag, '', items[item_index]['title'])
            link = items[item_index]['link']
            description = re.sub(remove_tag, '', items[item_index]['description'])
            blogger_name = items[item_index]['bloggername']
            blogger_link = items[item_index]['bloggerlink']            
            post_date = items[item_index]['postdate']                        
            blog_df.loc[idx] = [title, link, description, blogger_name,blogger_link,post_date]
            idx += 1
    else:
        print("Error Code:" + rescode)
blog_df.head(20)

검색 질의 :  애완견


,Title,Link,Description,Blogger Name,Blogger Link,postdate
0,애완견껌간식 치석제거에 스트레스까지 단번에 해결!,https://blog.naver.com/manpwup?Redirect=Log&lo...,"최대한 산책을 해주기는 하지만, 그걸로는 안되는 것 같아 애완견 껌간식을 주고 싶었...",ㅋ.ㅋ,https://blog.naver.com/manpwup,20211204
1,애완견사료 우리 강쥐 이제 딴 건 못먹어요.,https://blog.naver.com/guswlsl2867?Redirect=Lo...,"원래 급여하던 애완견사료를 먹이고 난 후로부터 눈물도 더 많이 나는 거 같고, 눈곱...",펫블리,https://blog.naver.com/guswlsl2867,20220607
2,"[꿈해몽] 개 꿈해몽, 개 꿈풀이, 애완견 꿈해몽, 강아지 나오는 꿈",https://photoguide.com/771,"애완견을 키우는 사람들이 가끔 자신이 기르는 강아지꿈을 꾸기도 하며, 또는 개를 키...",photoguide :: 포토가이드,https://photoguide.com/,20211122
3,광주애견분양 매력뿜뿜 애완견!,https://blog.naver.com/2236781?Redirect=Log&lo...,"푸들, 애완견 등이 있는데 저희 가족은 작은 체구에 까만코가 매력적인 말티즈에 반했...",봉구네 방앗간,https://blog.naver.com/2236781,20220531
4,애완견사료 추천 강아지 닥터독 피부모질사료,https://blog.naver.com/mel_holic?Redirect=Log&...,도저히 방법을 모르겠어서 여기저기 물어본 결과 애완견사료 추천 받았어요. 굉장히 배...,멜홀릭의 행복한 순간들,https://blog.naver.com/mel_holic,20211114
5,5사단 펜션 28사단 펜션 연천 플랫하우스 애완견 동반 가능해요,https://blog.naver.com/kujaka?Redirect=Log&log...,재미진 국내여행 섹션 5사단 펜션 28사단 펜션 연천 플랫하우스 애완견도 함께 할수...,❀ 구작가의 일상 ❀,https://blog.naver.com/kujaka,20220617
6,"목포 차박 좋은 곳 : 무안 수변공원 _ 운동, 애완견 산책하기...",https://blog.naver.com/vstar300?Redirect=Log&l...,"목포 차박 좋은 곳 수변공원 _ 운동, 애완견 산책하기 좋은 곳 요즘은 캠핑이 대세...",윤보리의 맛있는 여행,https://blog.naver.com/vstar300,20220623
7,성남강아지분양 고민하신다면 제일 좋은 애완견분양 매장에서...,https://blog.naver.com/morce1452?Redirect=Log&...,많은 애완견분양 매장이 있었기에 더 그랬던 것 같아요. 좋은 곳에서 가족을 만나고 ...,우당탕탕 삼남매,https://blog.naver.com/morce1452,20220430
8,영덕 강구항애완견동반 식당,https://blog.naver.com/leesoyul311?Redirect=Lo...,가성비는 물론 수율이 끝내주는 강구항애완견동반 식당에서 모두 박수치며 먹을 정도로 ...,서이의 먹방여행,https://blog.naver.com/leesoyul311,20220627
9,강릉여행 / 세인트존스 호텔 숙박후기 (애완견 동반가능호텔),https://blog.naver.com/tekart?Redirect=Log&log...,#세인트존스호텔 #강릉호텔 #강릉애완견동반호텔 #강릉바다호텔 #앤티가바부다의수도.....,여행은 동영상이지~,https://blog.naver.com/tekart,20211219


In [19]:
blog_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Title         1000 non-null   object
 1   Link          1000 non-null   object
 2   Description   1000 non-null   object
 3   Blogger Name  1000 non-null   object
 4   Blogger Link  1000 non-null   object
 5   postdate      1000 non-null   object
dtypes: object(6)
memory usage: 54.7+ KB


In [35]:
blog_df[['Title','postdate']]

,Title,postdate
0,반려동물 행동지도전공을 준비하고 있다면,20220622
1,셀럽스테이송도-테니스장가깝고 반려동물동반가능,20220619
2,반려동물관리사 자격증 준비방법 구체적으로,20220615
3,반려동물관리사 자격증 취득 방법 알기 쉽게,20220624
4,한검평 반려동물장례지도사 온라인 학습 과정,20220620
...,...,...
995,반려동물등록 페오펫 강아지 외장칩으로 등록완료,20220103
996,강원도 반려동물 동반 캠핑장 4곳 추천,20210528
997,반려동물 (feat 울 집 귀염둥이들),20220209
998,[참여요청]재난 시 반려동물도 함께 대피할 수 있게 법안 찬성...,20210928


In [31]:
blog_title = blog_df[['Title','postdate' ]]

In [38]:
blog_title.to_csv('./data/네이버api/블로그_(0627)naver_애완견.csv', encoding = 'utf-8')

# 네이버 쇼핑검색 api
https://developers.naver.com/docs/serviceapi/search/shopping/shopping.md#%EC%87%BC%ED%95%91

In [14]:
client_id = "--"
client_secret = "--"

query = urllib.parse.quote(input('검색 질의 : '))

idx = 0 # 번호남기기
display = 100 # 몇개씩보여줄래
start = 1
end = 1000 # 몇개씩가져올지
sort = 'sim'

shop_df = pd.DataFrame(columns = ('Title', 'Link', 'image', 'Low Price', 'High Price','Mall Name', 'Product ID', 'Product Type', 'Maker', 'Brand'))# 저장

for start_index in range(start, end, display): # 100개씩 가져온다.

    url = "https://openapi.naver.com/v1/search/shop?query=" + query +"&display="  + str(display) + "&start=" + str(start_index) + "&sort=" + sort  
        
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode==200): # 호출성공이200
        response_body = response.read()
        response_dict = json.loads(response_body.decode('utf-8'))
        items = response_dict['items']
        for item_index in range(0, len(items)):
            remove_tag = re.compile('<.*?>')
            title = re.sub(remove_tag, '', items[item_index]['title'])
            link = items[item_index]['link']
            image = items[item_index]['image']
            low_price = items[item_index]['lprice']
            high_price = items[item_index]['hprice']            
            mall_name = items[item_index]['mallName']            
            product_id = items[item_index]['productId']            
            product_type = items[item_index]['productType']                        
            maker = items[item_index]['maker']
            brand = items[item_index]['brand']
            
            shop_df.loc[idx] = [title, link, image, low_price, high_price, mall_name, product_id, product_type, maker, brand]
            idx += 1
    else:
        print("Error Code:" + rescode)
shop_df

검색 질의 : 댕댕


,Title,Link,image,Low Price,High Price,Mall Name,Product ID,Product Type,Maker,Brand
0,강아지 고양이 조끼 데일리 댕댕 냥이 옷 티,https://search.shopping.naver.com/gate.nhn?id=...,https://shopping-phinf.pstatic.net/main_279562...,10570,,네이버,27956271660,1,,
1,고양이 강아지 후드 티 냥 댕댕 후디 옷,https://search.shopping.naver.com/gate.nhn?id=...,https://shopping-phinf.pstatic.net/main_279703...,13420,,네이버,27970387453,1,,
2,반려동물 반려동물 토끼얼굴 평상복 냥이 댕댕,https://search.shopping.naver.com/gate.nhn?id=...,https://shopping-phinf.pstatic.net/main_279732...,14896,,네이버,27973200061,1,,
3,반려동물 댕댕 뉴 반려동물 후드 평상복 냥이,https://search.shopping.naver.com/gate.nhn?id=...,https://shopping-phinf.pstatic.net/main_308998...,9320,,네이버,30899895504,1,,
4,반려동물 댕댕 반려동물 냥이 베어버튼 평상복,https://search.shopping.naver.com/gate.nhn?id=...,https://shopping-phinf.pstatic.net/main_279542...,12720,,네이버,27954299284,1,,
...,...,...,...,...,...,...,...,...,...,...
995,오리고기 수제 강아지 간식 100g 1P 댕댕 애견 스낵,https://search.shopping.naver.com/gate.nhn?id=...,https://shopping-phinf.pstatic.net/main_319819...,5170,,롯데ON,31981946191,2,,
996,"💛댕댕반상 말랑말랑 쫀쫀 3색우유껌, 댕댕반상",https://search.shopping.naver.com/gate.nhn?id=...,https://shopping-phinf.pstatic.net/main_328089...,3850,,아이디어스idus,32808916425,2,,
997,오리고기 수제 강아지 간식 100g 1P 댕댕 애견 스낵,https://search.shopping.naver.com/gate.nhn?id=...,https://shopping-phinf.pstatic.net/main_840991...,4270,,모모랑비단,84099181062,2,펫스토리,펫스토리
998,"강아지 수제 간식 / 뼈 간식 / 캥거루 꼬리 80g, 댕댕수라상",https://search.shopping.naver.com/gate.nhn?id=...,https://shopping-phinf.pstatic.net/main_281258...,6500,,아이디어스idus,28125811291,2,,
